In [288]:
import re
import copy
import time
from collections import defaultdict

from IPython.display import display, HTML, clear_output
display(HTML("<style>.container { width:100% !important; }</style>"))

class Rock:
    def __init__(self, shape_type, height):
        self.shape_type = shape_type
        
        if shape_type == 'h':
            coords = [(x, 0) for x in range(2, 6)]
        elif shape_type == 'c':
            coords = [(2, 1), (3, 1), (3, 2), (3, 0), (4, 1)]
        elif shape_type == 'l':
            coords = [(2, 0), (3, 0), (4, 0), (4, 1), (4, 2)]
        elif shape_type == 'v':
            coords = [(2, 3), (2, 2), (2, 1), (2, 0)]
        elif shape_type == 's':
            coords = [(2, 1), (2, 0), (3, 1), (3, 0)]
            
        self.coords = set([(x, y + height + 3) for x, y in coords])
        
    def blow(self, direction):
        # right
        if direction == '>':
            return [(x+1, y) for x, y in self.coords]
            
        # left
        if direction == '<':
            return [(x-1, y) for x, y in self.coords]
        
    def fall(self):
        return [(x, y-1) for x, y in self.coords]
    
    def get_max_y(self):
        return max(y for x, y in self.coords)
        
    def move(self, coords):
        self.coords = coords
    
    def __str__(self):
        return f"{self.coords}"
    
    def __repr__(self):
        return self.__str__()
    
    def __contains__(self, coord):
        return coord in self.coords

class Chamber:
    def __init__(self, directions):
        self.width = 7
        self.directions = directions
        self.shape_types = ['h', 'c', 'l', 'v', 's']
        self.t = 0
        self.shapes = []
        self.coords = set()
        self.current_shape = None
        self.height = 0
        self.rock_count_to_height = {}
        
        # need these values to calculate big simulation heights
        self.m = defaultdict(list)
        self.cycle_start = None
        self.height_at_cycle_start = None
        self.cycle_length = None
        self.cycle_height = None
        self.height_at_remainder = None
        
    def get_shape_type_index(self):
        return len(self.shapes) % len(self.shape_types)
    
    def get_direction_index(self):
        return self.t % len(self.directions)
    
    def get_current_shape_type(self):
        return self.shape_types[self.get_shape_type_index()]
        
    def get_direction(self):
        return self.directions[self.get_direction_index()]
    
    def get_shape(self):
        shape_type = self.get_current_shape_type()
        return Rock(shape_type, self.height)
    
    def is_out_of_bounds(self, coords):
        for coord in coords:
            x, y = coord
            if x == -1 or x == self.width or y == -1:
                return True
            if coord in self.coords:
                return True
        return False
                
        
    def tick(self):
        if not self.current_shape:
            self.current_shape = self.get_shape()
            
        direction = self.get_direction()
        at_rest = False
        next_state = self.current_shape.blow(direction)
        if not self.is_out_of_bounds(next_state):
            self.current_shape.move(next_state)
            
        next_state = self.current_shape.fall()
        if not self.is_out_of_bounds(next_state):
            self.current_shape.move(next_state)
        else:
            at_rest = True
            
        if at_rest:
            self.shapes.append(self.current_shape)
            self.coords = self.coords.union(self.current_shape.coords)
            self.height = max(self.height, self.current_shape.get_max_y() + 1)
            self.current_shape = self.get_shape()

        self.t += 1
        
    def detect_cycle_start(self):

        i1, i2 = self.get_direction_index(), self.get_shape_type_index()
        
        self.m[(i1, i2)].append((self.height, len(self.shapes)))
        self.rock_count_to_height[len(self.shapes)] = self.height
        
        if len(self.m[(i1, i2)]) > 5:
            h1, r1 = self.m[(i1, i2)][-2]
            h2, r2 = self.m[(i1, i2)][-1]
            CYCLE_START = r1
            big = 1000000000000 - CYCLE_START
            HEIGHT_AT_CYCLE_START = h1
            CYCLE_LENGTH = r2 - r1
            CYCLE_HEIGHT = h2 - h1
            remainder = big % CYCLE_LENGTH
            HEIGHT_AT_REMAINDER_ROCK_COUNT = self.rock_count_to_height[CYCLE_START + remainder]
            HEIGHT_AT_REMAINDER = HEIGHT_AT_REMAINDER_ROCK_COUNT - HEIGHT_AT_CYCLE_START
            cycles = (big // CYCLE_LENGTH)

            print((cycles * CYCLE_HEIGHT) + HEIGHT_AT_CYCLE_START + HEIGHT_AT_REMAINDER)
            
        
        
    def __str__(self):
        output = ''
        for j in range(self.height + 5, -2, -1):
            for i in range(-1, self.width+1):
                if i == -1 or i == self.width:
                    output += '|'
                elif j == -1:
                    output += '-'
                elif (i, j) in self.current_shape:
                    output += '@'
                else:
                    ch = '.'
                    if (i, j) in self.coords:
                        ch = '#'
                    output += ch
            output += '\n'
            
        return output
        

with open('../inputs/17-sample.txt') as f:
    directions = f.read()
    print('len', len(directions))
    shapes = []

    chamber = Chamber(directions)
    rock_count = len(chamber.shapes) 
    # have to find when there is a "floor" in place as after that point
    # the simulation will start repeating on a predictable interval.
    cycle = 35
    threshold = 10000 # 70, 176, .., 388, 494, 600, 706 -- 53 every 35 
    value = 0
    cycle_values = [list() for i in range(cycle)]
    while rock_count < threshold:
        chamber.tick()
        chamber.detect_cycle_start()
        rock_count = len(chamber.shapes)

    
    

len 40
1400000000012
1533333333334
1666666666656
1599999999995
1599999999995
1599999999995
1599999999995
1599999999995
1599999999995
1733333333317
1733333333317
1733333333317
1733333333317
1599999999997
1599999999997
1733333333317
1599999999997
1599999999997
1599999999997
1599999999997
1466666666677
1599999999997
1500000000007
1500000000007
1500000000007
1400000000017
1400000000017
1400000000017
1400000000017
1400000000017
1466666666676
1466666666676
1533333333335
1533333333335
1466666666676
1450000000012
1450000000012
1450000000012
1450000000012
1450000000012
1450000000012
1350000000022
1350000000022
1350000000022
1350000000022
1350000000022
1350000000022
1350000000022
1400000000017
1533333333335
1533333333335
1533333333335
1450000000010
1450000000010
1450000000010
1450000000010
1666666666653
1549999999998
1599999999994
1599999999994
1599999999994
1599999999994
1599999999994
1599999999994
1733333333310
1733333333310
1733333333310
1733333333310
1733333333310
1733333333310
1599999999992

1514285714288
1514285714288
1514285714288
1514285714288
1400000000257
1514285714288
1514285714288
1514285714288
1533333333295
1533333333295
1533333333295
1514285714288
1514285714288
1514285714288
1450000000145
1450000000145
1450000000145
1450000000145
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1666666666333
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1549999999923
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1599999999814
1599999999814
1599999999814
1599999999814
1599999999814
1599999999814
1514285714288
1514285714288
1733333332850
1733333332850
1733333332850
1733333332850
1733333332850
173333

1599999999716
1599999999716
1599999999716
1599999999716
1599999999716
1599999999716
1514285714288
1514285714288
1733333332604
1733333332604
1733333332604
1733333332604
1733333332604
1733333332604
1599999999716
1599999999716
1599999999716
1733333332604
1514285714288
1514285714288
1514285714288
1514285714288
1599999999716
1514285714288
1514285714288
1514285714288
1549999999883
1549999999883
1514285714288
1514285714288
1500000000050
1500000000050
1514285714288
1514285714288
1514285714288
1549999999883
1599999999718
1599999999718
1599999999718
1599999999718
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1466666666832
1514285714288
1514285714288
1514285714288
1514285714288
1599999999719
1514285714288
1514285714288
1514285714288
1500000000055
150000

1514285714288
1514285714288
1514285714288
1549999999855
1549999999855
1514285714288
1514285714288
1500000000066
1500000000066
1514285714288
1514285714288
1514285714288
1549999999855
1599999999643
1599999999643
1599999999643
1599999999643
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1466666666873
1514285714288
1514285714288
1514285714288
1514285714288
1599999999644
1514285714288
1514285714288
1514285714288
1500000000066
1500000000066
1500000000066
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1400000000490
140000

1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1599999999580
1599999999580
1599999999580
1599999999580
1599999999580
1599999999580
1514285714288
1514285714288
1733333332252
1733333332252
1733333332252
1733333332252
1733333332252
1733333332252
1599999999580
1599999999580
1599999999580
1733333332252
1514285714288
1514285714288
1514285714288
1514285714288
1599999999580
1514285714288
1514285714288
1514285714288
1549999999828
1549999999828
1514285714288
1514285714288
1500000000076
1500000000076
1514285714288
1514285714288
1514285714288
1549999999828
1599999999580
1599999999580
1599999999580
1599999999580
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
151428

...

1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1599999999311
1599999999311
1599999999311
1599999999311
1599999999311
1599999999311
1514285714288
1514285714288
1733333331569
1733333331569
1733333331569
1733333331569
1733333331569
1733333331569
1599999999313
1599999999313
1599999999313
1733333331569
1514285714288
1514285714288
1514285714288
1514285714288
1599999999313
1514285714288
1514285714288
1514285714288
1549999999717
1549999999717
1514285714288
1514285714288
1500000000121
1500000000121
1514285714288
1514285714288
1514285714288
1549999999717
1599999999313
1599999999313
1599999999313
1599999999313
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1466666667057
151428

1533333333175
1533333333175
1514285714288
1514285714288
1514285714288
1466666667076
1514285714288
1514285714288
1514285714288
1514285714288
1450000000552
1450000000552
1450000000552
1450000000552
1450000000552
1450000000552
1514285714288
1514285714288
1350000001402
1350000001402
1350000001402
1350000001402
1350000001402
1350000001402
1514285714288
1514285714288
1514285714288
1350000001402
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1400000000977
1514285714288
1514285714288
1514285714288
1533333333175
1533333333175
1533333333175
1514285714288
1514285714288
1514285714288
1450000000550
1450000000550
1450000000550
1450000000550
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1666666665373
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
154999

1514285714288
1514285714288
1733333331378
1733333331378
1733333331378
1733333331378
1733333331378
1733333331378
1599999999236
1599999999236
1599999999236
1733333331378
1514285714288
1514285714288
1514285714288
1514285714288
1599999999236
1514285714288
1514285714288
1514285714288
1549999999683
1549999999683
1514285714288
1514285714288
1500000000130
1500000000130
1514285714288
1514285714288
1514285714288
1549999999683
1599999999238
1599999999238
1599999999238
1599999999238
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1466666667098
1514285714288
1514285714288
1514285714288
1514285714288
1599999999239
1514285714288
1514285714288
1514285714288
1500000000135
1500000000135
1500000000135
1514285714288
1514285714288
1514285714288
1514285714288
151428

1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1599999999202
1599999999202
1599999999202
1599999999202
1599999999202
1599999999202
1514285714288
1514285714288
1733333331286
1733333331286
1733333331286
1733333331286
1733333331286
1733333331286
1599999999200
1599999999200
1599999999200
1733333331286
1514285714288
1514285714288
1514285714288
1514285714288
1599999999200
1514285714288
1514285714288
1514285714288
1549999999668
1549999999668
1514285714288
1514285714288
1500000000136
1500000000136
1514285714288
1514285714288
1514285714288
1549999999668
1599999999202
1599999999202
1599999999202
1599999999202
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1466666667118
1514285714288
1514285714288
1514285714288
1514285714288
159999

1514285714288
1514285714288
1514285714288
1514285714288
1666666665181
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1549999999653
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1514285714288
1599999999166
1599999999166
1599999999166
1599999999166
1599999999166
1599999999166
1514285714288
1514285714288
1733333331194
1733333331194
1733333331194
1733333331194
1733333331194
1733333331194
1599999999164
1599999999164
1599999999164
1733333331194
1514285714288
1514285714288
1514285714288
1514285714288
1599999999164
1514285714288
1514285714288
1514285714288
1549999999653
1549999999653
1514285714288
1514285714288
1500000000142
1500000000142
1514285714288
1514285714288
1514285714288
1549999999653
1599999999166
1599999999166
1599999999166
1599999999166
151428

In [ ]:
# 623876305776 too low
# 1602881845146 too high
# 1602881844347